# SWOT PIXC pre processing

In [1]:
import xarray as xr
import pandas as pd
import numpy as np

### One file test

In [17]:
# ---------- Filepath ----------
aufeis_test_tile_filepath = "../../../../shared_space/SWOT_Aufeis/SWOT_PIXC_data/SWOT_L2_HR_PIXC_033_487_278R_20250605T070551_20250605T070602_PID0_01.nc"

# NetCDF structure check
import netCDF4
data_structure = netCDF4.Dataset(aufeis_test_tile_filepath)
# print(data_structure) # shows 'groups: pixel_cloud, tvp, noise'

# file pointer
fp = xr.open_dataset(aufeis_test_tile_filepath, group = 'pixel_cloud')
# print(fp)
# print(fp.attrs)

(4423064, 14)
    longitude   latitude  phase_noise_std  waterfrac  geolocqual        sig0  \
0 -149.552404  69.236015         0.045659   0.167256           0   24.465372   
1 -149.551882  69.236147         0.031159   1.224904           0  322.846558   
2 -149.551167  69.236328         0.026563   2.583732           0  965.866943   
3 -149.550266  69.236555         0.022460   1.348676           0  588.309814   
4 -149.549266  69.236808         0.023962   0.945739           0  326.267853   

   sig0_uncert    crosstrack   elevation  cycle_number  pass_number  \
0    11.847507 -13393.489258  296.291781            35           52   
1   151.669876 -13441.602539  295.972732            35           52   
2   452.993774 -13492.628906  295.815024            35           52   
3   276.064575 -13541.678711  295.813125            35           52   
4   153.266998 -13596.972656  295.896783            35           52   

   tile_number           time_granule_start  \
0           30  2025-07-01T10:5

In [14]:
# Extract the variables we want into 1D numpy arrays
def getvar(name):
    return fp[name].values.ravel() if name in fp else np.nan

# Build a Pandas DataFrame
SWOT_Points = pd.DataFrame({
    "longitude": getvar("longitude"),
    "latitude": getvar("latitude"),
    "height": getvar("height"),
    "phase_noise_std": getvar("phase_noise_std"),
    "dheight": getvar("dheight_dphase"),
    "class": getvar("classification"),
    "classqual": getvar("classification"),
    "bright_land_flag": getvar("bright_land_flag"),
    "ancillary_surface_classification_flag": getvar("ancillary_surface_classification_flag"),
    "waterfrac": getvar("water_frac"),
    "waterfrac_uncert": getvar("water_frac_uncert"),
    "prior_water_prob": getvar("prior_water_prob"),
    "geolocqual": getvar("geolocation_qual"),
    "sig0": getvar("sig0"),
    "sig0_uncert": getvar("sig0_uncert"),
    "sig0_qual": getvar("sig0_qual"),
    "crosstrack": getvar("cross_track"),
    "pixel_area": getvar("pixel_area"),
    "darea_dheight": getvar("darea_dheight"),
    # elevation corrections
    "geoid": getvar("geoid"),
    "solid_tide": getvar("solid_earth_tide"),
    "load_tide": getvar("load_tide_fes"),
    "pole_tide": getvar("pole_tide")
})

# Derive elevation & height uncertainty
SWOT_Points["height_uncert"] = SWOT_Points["phase_noise_std"] * SWOT_Points["dheight"]
SWOT_Points["geoid_correction"] = SWOT_Points["geoid"] - SWOT_Points["solid_tide"] - SWOT_Points["load_tide"] - SWOT_Points["pole_tide"]
SWOT_Points["elevation"] = SWOT_Points["height"] - SWOT_Points["geoid_correction"]

# Drop any empty rows
SWOT_Points = SWOT_Points.dropna(how="all")

# Quality filtering
geolocqual_problem_bits = {
    4, 4101, 5, 6, 4100, 4102, 524292, 524293, 524294, 524295,
    528389, 528390, 7, 528388, 16777220, 17301508, 17305604,
    528391, 4103
}

SWOT_Points = SWOT_Points[
    (~SWOT_Points["geolocqual"].isin(geolocqual_problem_bits)) &
    (SWOT_Points["crosstrack"].abs().between(10000, 60000))]

print(SWOT_Points.head())
fp.close()

      longitude   latitude      height  phase_noise_std   dheight  class  \
211 -143.098029  68.201372  571.506653         0.117613  0.744377    6.0   
212 -143.096199  68.200914  571.323730         0.110314  0.758199    6.0   
213 -143.093815  68.200317  571.329895         0.131377  0.776198    3.0   
285 -142.934672  68.160307  572.359375         0.109845  1.980290    3.0   
286 -142.934076  68.160156  572.105469         0.061149  1.984812    3.0   

     classqual  bright_land_flag  ancillary_surface_classification_flag  \
211        6.0               0.0                                    1.0   
212        6.0               0.0                                    1.0   
213        3.0               0.0                                    1.0   
285        3.0               0.0                                    1.0   
286        3.0               0.0                                    1.0   

     waterfrac     ...        crosstrack   pixel_area  darea_dheight  \
211   0.888415     .

In [15]:
# Need to add metadata attributes

# Get the attributes of interest from the data_structure
cycle_number = data_structure.cycle_number
pass_number = data_structure.pass_number
tile_number = data_structure.tile_number
time_granule_start = data_structure.time_granule_start

# Add these as new columns to the SWOT_Points DataFrame
SWOT_Points["cycle_number"] = cycle_number
SWOT_Points["pass_number"] = pass_number
SWOT_Points["tile_number"] = tile_number
SWOT_Points["time"] = time_granule_start # just taking start time since it's a diff of seconds

In [ ]:
# need to add cycle, pass, time to SWOT_Points df
# for ML:
# longitude, latitude, NORMALIZED to each tile elevation, phase_noise_std, sig0
# crosstrack, maybe cycle & pass??

### Batch processing with initial date filtering

In [3]:
import os
import glob
import re
import traceback
from datetime import datetime, timedelta

import netCDF4
import numpy as np
import pandas as pd

# for fancy progress bars
from tqdm import tqdm

In [4]:
# ---------------------------------------------------------------------------------------------------------------
# Sentinel-2 dates

import pandas as pd
from datetime import datetime

# File path
csv_file_path = 'data/image_dates.csv'

# Read CSV and parse 'date' column as datetime
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Filter to dates after July 1, 2025
cutoff_date = datetime(2025, 7, 1)
date_objs = df[df['date'] > cutoff_date]['date'].tolist()

day_tolerance = 1

# Display results
print(len(date_objs))

35


In [5]:
# ---------------------------
# setup
# ---------------------------
base_dir = "../../../../shared_space/SWOT_Aufeis/SWOT_PIXC_data"
pattern = os.path.join(base_dir, "SWOT_L2_HR_PIXC_*.nc")

# regex to find timestamps like 20250605T070551 in filenames
timestamp_re = re.compile(r"\d{8}T\d{6}")

# geolocqual problem bits
geolocqual_problem_bits = {
    4, 4101, 5, 6, 4100, 4102, 524292, 524293, 524294, 524295,
    528389, 528390, 7, 528388, 16777220, 17301508, 17305604,
    528391, 4103
}

# mapping from netCDF variable names to DataFrame column names
mapping = {
    "longitude": "longitude",
    "latitude": "latitude",
    "height": "height",
    "phase_noise_std": "phase_noise_std",
#     "dheight_dphase": "dheight",
#     "classification": "classification",
#     "bright_land_flag": "bright_land_flag",
#     "ancillary_surface_classification_flag": "ancillary_surface_classification_flag",
    "water_frac": "waterfrac",
#     "water_frac_uncert": "waterfrac_uncert",
#     "prior_water_prob": "prior_water_prob",
    "geolocation_qual": "geolocqual",
    "sig0": "sig0",
    "sig0_uncert": "sig0_uncert",
#     "sig0_qual": "sig0_qual",
    "cross_track": "crosstrack",
#     "pixel_area": "pixel_area",
#     "darea_dheight": "darea_dheight",
    "geoid": "geoid",
    "solid_earth_tide": "solid_tide",
    "load_tide_fes": "load_tide",
    "pole_tide": "pole_tide"
}

# Columns to drop computing elevation (attempting to reduce written file size & memory here)
cols_to_drop_after_elevation = [
    "geoid_correction", "pole_tide", "load_tide", "solid_tide", "geoid", "height"
]

In [6]:
# ---------------------------
# Helpers
# ---------------------------
def safe_var_read(group, varname):
    """Read a variable from a netCDF4 group safely. Return numpy array or None."""
    if varname not in group.variables:
        return None
    var = group.variables[varname]
    arr = var[:]
    if np.ma.isMaskedArray(arr):
        arr = arr.filled(np.nan)
    return np.asarray(arr)


def process_single_file(nc_path, geolocqual_problem_bits=geolocqual_problem_bits):
    """
    Memory-efficient processing: read small variables first, compute mask,
    then read only masked indices for other variables.
    Returns a DataFrame (or None on error / empty).
    """
    try:
        with netCDF4.Dataset(nc_path, mode="r") as ds_root:
            # access pixel_cloud group if present
            grp = ds_root.groups.get("pixel_cloud", ds_root)

            # read minimal variables for mask
            geolocqual = safe_var_read(grp, "geolocation_qual")
            crosstrack = safe_var_read(grp, "cross_track")

            if geolocqual is None or crosstrack is None:
                print(f"Missing geolocqual or crosstrack in {nc_path}; skipping file.")
                return None

            geolocqual = np.asarray(geolocqual).ravel()
            crosstrack = np.asarray(crosstrack).ravel()

            # prepare integer version of geolocqual (NaNs -> sentinel)
            valid_mask_geo = ~np.isnan(geolocqual)
            geolocqual_int = np.full(geolocqual.shape, -999999, dtype=np.int64)
            if valid_mask_geo.any():
                geolocqual_int[valid_mask_geo] = geolocqual[valid_mask_geo].astype(np.int64)

            mask_geoloc = ~np.isin(geolocqual_int, list(geolocqual_problem_bits))

            abs_ct = np.abs(crosstrack)
            mask_crosstrack = (abs_ct >= 10000) & (abs_ct <= 60000)

            good_mask = mask_geoloc & mask_crosstrack

            if not np.any(good_mask):
                return None

            n_good = int(np.count_nonzero(good_mask))
            data = {}

            # read each variable but only keep good_mask indices (defensive about missing vars)
            for varname, colname in mapping.items():
                if varname == "geolocation_qual":
                    data[colname] = geolocqual[good_mask]
                    continue
                if varname == "cross_track":
                    data[colname] = crosstrack[good_mask]
                    continue

                arr = safe_var_read(grp, varname)
                if arr is None:
                    data[colname] = np.full(n_good, np.nan)
                else:
                    arr = np.asarray(arr).ravel()
                    if arr.shape[0] < good_mask.shape[0]:
                        tmp = np.full(good_mask.shape[0], np.nan)
                        tmp[: arr.shape[0]] = arr
                        arr = tmp
                    data[colname] = arr[good_mask]

            # build dataframe and derived columns
            SWOT_df = pd.DataFrame(data)

            geoid_vals = np.asarray(SWOT_df["geoid"].values, dtype=float)
            solid_tide_vals = np.asarray(SWOT_df["solid_tide"].values, dtype=float)
            load_tide_vals = np.asarray(SWOT_df["load_tide"].values, dtype=float)
            pole_tide_vals = np.asarray(SWOT_df["pole_tide"].values, dtype=float)
            SWOT_df["geoid_correction"] = geoid_vals - solid_tide_vals - load_tide_vals - pole_tide_vals

            height_vals = np.asarray(SWOT_df["height"].values, dtype=float)
            SWOT_df["elevation"] = height_vals - SWOT_df["geoid_correction"].values
            
            # ---- DROP heavy/intermediate columns BEFORE writing ----
            for c in cols_to_drop_after_elevation:
                if c in SWOT_df.columns:
                    del SWOT_df[c]

            SWOT_df = SWOT_df.dropna(how="all")
            if SWOT_df.empty:
                return None

            # attach root-level metadata
            def safe_attr(ds, name):
                return getattr(ds, name, np.nan)

            SWOT_df["cycle_number"] = safe_attr(ds_root, "cycle_number")
            SWOT_df["pass_number"] = safe_attr(ds_root, "pass_number")
            SWOT_df["tile_number"] = safe_attr(ds_root, "tile_number")
            SWOT_df["time_granule_start"] = safe_attr(ds_root, "time_granule_start")
            SWOT_df["source_file"] = os.path.basename(nc_path)

            return SWOT_df

    except Exception as e:
        print(f"Error processing {nc_path}: {e}")
        traceback.print_exc()
        return None

In [7]:
# ---------------------------
# Main!!! find files, extract timestamps, filter, process
# ---------------------------
def main():
    # find all files
    all_files = sorted(glob.glob(pattern))
    print(f"Found {len(all_files)} files matching pattern.")

    # extract timestamps from filenames
    file_to_ts = {}
    for f in all_files:
        m = timestamp_re.search(os.path.basename(f))
        if m:
            ts = m.group()  # e.g., "20250605T070551"
            file_to_ts[f] = ts

    # filter timestamps to those within +/- day_tolerance of any target date
    filtered_timestamps = []
    for ts in sorted(set(file_to_ts.values())):
        ts_date = datetime.strptime(ts[:8], "%Y%m%d")
        if any(abs((ts_date - d).days) <= day_tolerance for d in date_objs):
            filtered_timestamps.append(ts)

    print(f"Found {len(filtered_timestamps)} timestamps within +/-{day_tolerance} days of targets.")
    if filtered_timestamps:
        print("Filtered timestamps (sample):", filtered_timestamps[:10])

    # filter files to only those containing a kept timestamp
    filtered_files = [f for f, ts in file_to_ts.items() if ts in filtered_timestamps]
    print(f"Filtered to {len(filtered_files)} files to process.")

    # optional: show unique days in filtered_timestamps
    unique_days = sorted({ts[:8] for ts in filtered_timestamps})
    print("Unique days in filtered timestamps:", unique_days)
    print("Total unique days:", len(unique_days))
            
    dfs = []
    for i, f in enumerate(tqdm(filtered_files), 1):
        print(f"\n[{i}/{len(filtered_files)}] Processing: {os.path.basename(f)}")
        df = process_single_file(f)
        if df is not None and not df.empty:
            dfs.append(df)
            print(f"Completed: {os.path.basename(f)} ({len(df)} rows)")
        else:
            print(f"Skipping empty/failed result for {os.path.basename(f)}")

    if dfs:
        SWOT_Points_all = pd.concat(dfs, ignore_index=True)
        print("Combined DataFrame shape:", SWOT_Points_all.shape)
    else:
        SWOT_Points_all = pd.DataFrame()
        print("No valid data found in filtered files.")

    # return the DataFrame and some metadata for further use
    return {
        "SWOT_Points_all": SWOT_Points_all,
        "filtered_files": filtered_files,
        "filtered_timestamps": filtered_timestamps,
        "unique_days": unique_days
    }


if __name__ == "__main__":
    results = main()
    # proof that it worked plz
    if not results["SWOT_Points_all"].empty:
        print(results["SWOT_Points_all"].head())

  0%|          | 0/36 [00:00<?, ?it/s]

Found 135 files matching pattern.
Found 33 timestamps within +/-1 days of targets.
Filtered timestamps (sample): ['20250701T105509', '20250706T021314', '20250707T021355', '20250717T003602', '20250718T003643', '20250718T003653', '20250719T003714', '20250719T003724', '20250719T003734', '20250719T073822']
Filtered to 36 files to process.
Unique days in filtered timestamps: ['20250701', '20250706', '20250707', '20250717', '20250718', '20250719', '20250720', '20250721', '20250726', '20250727', '20250728', '20250731', '20250801', '20250806', '20250807', '20250808', '20250811', '20250812', '20250816', '20250817']
Total unique days: 20

[1/36] Processing: SWOT_L2_HR_PIXC_035_052_030L_20250701T105509_20250701T105520_PID0_01.nc


  3%|▎         | 1/36 [00:00<00:28,  1.24it/s]

Completed: SWOT_L2_HR_PIXC_035_052_030L_20250701T105509_20250701T105520_PID0_01.nc (197238 rows)

[2/36] Processing: SWOT_L2_HR_PIXC_035_181_278L_20250706T021314_20250706T021325_PID0_01.nc


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in greater_equal
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in less_equal
  6%|▌         | 2/36 [00:01<00:23,  1.43it/s]

...

 97%|█████████▋| 35/36 [00:21<00:00,  1.58it/s]

Completed: SWOT_L2_HR_PIXC_037_181_278L_20250816T194323_20250816T194334_PID0_01.nc (81799 rows)

[36/36] Processing: SWOT_L2_HR_PIXC_037_209_279R_20250817T194404_20250817T194415_PID0_01.nc


100%|██████████| 36/36 [00:21<00:00,  1.90it/s]

Completed: SWOT_L2_HR_PIXC_037_209_279R_20250817T194404_20250817T194415_PID0_01.nc (61006 rows)


Combined DataFrame shape: (4423064, 14)
    longitude   latitude  phase_noise_std  waterfrac  geolocqual        sig0  \
0 -149.552404  69.236015         0.045659   0.167256           0   24.465372   
1 -149.551882  69.236147         0.031159   1.224904           0  322.846558   
2 -149.551167  69.236328         0.026563   2.583732           0  965.866943   
3 -149.550266  69.236555         0.022460   1.348676           0  588.309814   
4 -149.549266  69.236808         0.023962   0.945739           0  326.267853   

   sig0_uncert    crosstrack   elevation  cycle_number  pass_number  \
0    11.847507 -13393.489258  296.291781            35           52   
1   151.669876 -13441.602539  295.972732            35           52   
2   452.993774 -13492.628906  295.815024            35           52   
3   276.064575 -13541.678711  295.813125            35           52   
4   153.266998 -13596.972656  295.896783            35           52   

   tile_number           time_granule_start  \
0    

In [15]:
# save as a csv

output_csv = "../../../../shared_space/SWOT_Aufeis/SWOT_PIXC_data/filtered_SWOT_PIXC.csv"

results["SWOT_Points_all"].to_csv(output_csv, index=False)

In [16]:
# save as a python pickle

import pickle
output_pkl = "../../../../shared_space/SWOT_Aufeis/SWOT_PIXC_data/filtered_SWOT_PIXC.pkl"

results["SWOT_Points_all"].to_pickle(output_pkl)

# this is MUCH faster